# Lesson 2.1 多分类策略——OvO、OvR、MvM

In [1]:
# 科学计算模块
import numpy as np
import pandas as pd

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

- 多分类问题描述

&emsp;&emsp;当离散型标签拥有两个以上分类水平时，即对多个（两个以上）分类进行类别预测的问题，被称为多分类问题。例如有如下四分类问题简单数据集：

<center><img src="https://i.loli.net/2021/04/19/PFGf1qrXkp7t39g.jpg" alt="80" style="zoom:33%;" />

其中index是每条数据编号，labels是每条数据的标签。

- 多分类问题解决思路

&emsp;&emsp;一般来说，用二分类学习器解决多分类问题，基本思想是先拆分后集成，也就是先将数据集进行拆分，然后多个数据集可训练多个模型，然后再对多个模型进行集成。这里所谓集成，指的是使用这多个模型对后续新进来数据的预测方法。

&emsp;&emsp;具体来看，依据该思路一般有三种实现策略，分别是“一对一”（One vs Ons，简称OvO）、“一对剩余”（One vs Rest，简称OvR）和“多对多”（Many vs Many，加成MvM）。接下来我们逐个讨论。

### 1.OvO策略

- 拆分策略

&emsp;&emsp;OvO的拆分策略比较简单，基本过程是将每个类别对应数据集单独拆分成一个子数据集，然后令其两两组合，再来进行模型训练。例如，对于上述四分类数据集，根据标签类别可将其拆分成四个数据集，然后再进行两两组合，总共有6种组合，也就是$C^2_4$种组合。拆分过程如下所示：

<center><img src="https://i.loli.net/2021/04/19/NT8MEoSJcdFaVL9.jpg" alt="81" style="zoom:33%;" />

然后在这6个新和成的数据集上，我们就能训练6个分类器。当然，如果是N分类问题，则需要训练$C^2_N=\frac{N(N-1)}{2}$个模型。

- 集成策略

&emsp;&emsp;当模型训练完成之后，接下来面对新数据集的预测，可以使用投票法从6个分类器的判别结果中挑选最终判别结果。

<center><img src="https://i.loli.net/2021/04/19/8ouyVcgx2j4s3AJ.jpg" alt="82" style="zoom:33%;" />

根据少数服从多数的投票法能够得出，某条新数据最终应该属于类别1。

### 2.OvR策略

- 拆分策略

&emsp;&emsp;和OvO的两两组合不同，OvR策略则是每次将一类的样例作为正例、其他所有数据作为反例来进行数据集拆分。对于上述四分类数据集，OvR策略最终会将其拆分为4个数据集，基本拆分过程如下：

<center><img src="https://i.loli.net/2021/04/19/t9fKCdqOnwgJmEM.jpg" alt="83" style="zoom:33%;" />

此4个数据集就将训练4个分类器。注意，在OvR的划分策略种，是将rest无差别全都划分为负类。当然，如果数据集总共有N个类别，则在进行数据集划分时总共将拆分成N个数据集。

- 集成策略

&emsp;&emsp;当成，集成策略和划分策略息息相关，对于OvR方法来说，对于新数据的预测，如果仅有一个分类器将其预测为正例，则新数据集属于该类。若有多个分类器将其预测为正例，则根据分类器本身准确率来进行判断，选取准确率更高的那个分类器的判别结果作为新数据的预测结果。

<center><img src="https://i.loli.net/2021/04/19/Lp8dYKfSRuTMnZ3.jpg" alt="84" style="zoom:33%;" />

- OvO和OvR的比较

&emsp;&emsp;对于这两种策略来说，尽管OvO需要训练更多的基础分类器，但由于OvO中的每个切分出来的数据集都更小，因此基础分类器训练时间也将更短。因此，综合来看在训练时间开销上，OvO往往要小于OvR。而在性能方面，大多数情况下二者性能类似。

### 3.MvM策略

&emsp;&emsp;相比于OvO和OvR，MvM是一种更加复杂的策略。MvM要求同时将若干类化为正类、其他类化为负类，并且要求多次划分，再进行集成。一般来说，通常会采用一种名为“纠错输入码”（Error Correcting Output Codes，简称ECOC）的技术来实现MvM过程。

- 拆分策略

&emsp;&emsp;此时对于上述4分类数据集，拆分过程就会变得更加复杂，我们可以任选其中一类作为正类、其余作为负类，也可以任选其中两类作为正类、其余作为负数，以此类推。由此则诞生出了非常多种子数据集，对应也将训练非常多个基础分类器。

> 当然，将某一类视作正类和将其余三类视作正类的预测结果相同，对调下预测结果即可，此处不用重复划分。

例如，对于上述4分类数据集，则可有如下划分方式：

<center><img src="https://i.loli.net/2021/04/19/rCJLkghq5ROj1p6.jpg" alt="85" style="zoom:33%;" />

根据上述划分方式，总共将划分$C_4^1+C_4^2=10$个数据集，对应构建，对应的我们可以构建10个分类器。不过一般来说对于ECOC来说我们不会如此详尽的对数据集进行划分，而是再上述划分结果中挑选部分数据集进行建模，例如就挑选上面显式表示的4个数据集来进行建模，即可 构建4个分类器。

> 由此我们也不难看出OvR实际上是MvM的一种特例。

- 集成策略

&emsp;&emsp;接下来我们进行模型集成。值得注意的是，如果是以上述方式划分四个数据集，我们可以将每次划分过程中正例或负例的标签所组成的数组视为每一条数据自己的编码。如下所示：

<center><img src="https://i.loli.net/2021/04/19/v1AN2FZztLpVYbl.jpg" alt="86" style="zoom:33%;" />

同时，我们使用训练好的四个基础分类器对新数据进行预测，也将产生四个结果，而这四个结果也可构成一个四位的新数据的编码。接下来，我们可以计算新数据的编码和上述不同类别编码之间的距离，从而判断新生成数据应该属于哪一类。

<center><img src="https://i.loli.net/2021/04/19/hydDjczHIvS8EsV.jpg" alt="87" style="zoom:33%;" />

> 不拿发现，如果预测足够准确，编码其实是和类别一一对应的。但如果基础分类器预测类别不够准确，编码和类别并不一定会一一对应。

当然，距离计算有很多种方法，此处简单进行介绍，假设$x$和$y$是两组n维数据如下所示：

$$x=(x_1, x_2, ..., x_n)$$

$$y=(y_1,y_2,...,y_n)$$

则**欧式距离**计算公式如下：

$$d(x, y) = \sqrt{\sum_{i = 1}^{n}(x_i-y_i)^2}$$

即对应位置元素依次相减后取其平方和再开平方。

**街道距离**计算公式如下：

$$d(x, y) =\sum_{i = 1}^{n}(|x_i-y_i|)$$

即对应位置元素依次相减后取其绝对值的和。

**闵可夫斯基距离**计算公式如下：

$$d(x, y) = \sqrt[n]{\sum_{i = 1}^{n}(|x_i-y_i|)^n}$$

不难发现，其实街道距离和欧式距离都是闵可夫斯基距离的特例。

&emsp;&emsp;此处以欧式距离为例计算新数据编码和各类编码之间距离。为了方便运算，此处可定义闵可夫斯基距离计算函数如下：

In [11]:
def dist(x, y, cat = 2):
    """
    闵可夫斯基距离计算函数(默认为街道距离)
    """
    d1 = np.abs(x - y)
    if x.ndim > 1 or y.ndim > 1:
        res1 = np.power(d1, cat).sum(1)
    else:
        res1 = np.power(d1, cat).sum()
    res = np.power(res1, 1/cat)
    return res

验证函数性能：

In [3]:
x = np.array([1, 2])
y = np.array([2, 3])

In [4]:
dist(x, y)

1.4142135623730951

In [5]:
np.sqrt(2)

1.4142135623730951

进行编码距离计算：

In [6]:
# 原类别编码矩阵
code_mat = np.array([[1, -1, 1, -1],
                     [-1, -1, 1, -1],
                     [-1, -1, -1, 1],
                     [-1, 1, -1, 1]])

# 预测数据编码
data_code = np.array([1, -1, 1, 1])

In [12]:
dist(code_mat, data_code)  #街道距离

array([2.        , 2.82842712, 2.82842712, 3.46410162])

也可通过以下方式验证：

In [8]:
np.abs(code_mat - data_code) 

array([[0, 0, 0, 2],
       [2, 0, 0, 2],
       [2, 0, 2, 0],
       [2, 2, 2, 0]])

In [13]:
np.sqrt(np.power(np.abs(code_mat - data_code), 2).sum(1))

array([2.        , 2.82842712, 2.82842712, 3.46410162])

不难发现，新样本应该属于第一类。至此，我们就完成了MvM的一次多分类预测的全流程。

- ECOC方法评估

&emsp;&emsp;对于ECOC方法来说，编码越长预测结果越准确，不过编码越长也代表着需要耗费更多的计算资源，并且由于模型本身类别有限，因此数据集划分数量有限，编码长度也会有限。不过一般来说，相比OvR，MvM方法效果会更好。